In [1]:
import pandas as pd

df = pd.read_csv("test.csv")
df

,Column1,llm_nct_id,llm_eligibility_criteria,llm_ncit_code,llm_type,llm_extracted_term,llm_selected_term,llm_selection_proximity,llm_eligibility_criterion,llm_invalid_ncit_code,...,ctsapi_ncit_code,ctsapi_type,ctsapi_term,ctsapi_eligibility_criterion,ctsapi_branch,ctsapi_expected_branch_code,ctsapi_expected_branch_type,ctsapi_expected_branch_name,ctsapi_type_matches_branch_type,ctsapi_branch_matches_branch_name
0,0,NCT02991651,Inclusion Criteria:\n* Patients must have hist...,C107553,biomarkers,AST(SGOT)/ALT(SGPT) ≤2.5 × ULN or ≤5 x ULN if ...,ALP,closest,inclusion,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NCT02991651,Inclusion Criteria:\n* Patients must have hist...,C104196,biomarkers,"Absolute neutrophil count ≥1,500/mcL","Elastase, Neutrophil Gene",closest,inclusion,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NCT02991651,Inclusion Criteria:\n* Patients must have hist...,C106347,biomarkers,Activating EGFR mutations,EGF Containing Fibulin Like Extracellular Matr...,closest,exclusion,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NCT02991651,Inclusion Criteria:\n* Patients must have hist...,C103382,biomarkers,Creatinine clearance ≥40 mL/min,Creatinine Clearance,closest,inclusion,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NCT02991651,Inclusion Criteria:\n* Patients must have hist...,C131567,biomarkers,Hemoglobin ≥8.5 g/dL,HBBD,closest,inclusion,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,NCT02991651,Inclusion Criteria:\n* Patients must have hist...,C104196,biomarkers,"Leukocytes ≥3,000/mcL","Elastase, Leukocyte Gene",closest,inclusion,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,NCT02991651,Inclusion Criteria:\n* Patients must have hist...,C126550,biomarkers,"Platelets ≥100,000/mcL",Glycoprotein Ib Platelet Beta Subunit wt Allele,closest,inclusion,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,NCT02991651,Inclusion Criteria:\n* Patients must have hist...,C105863,biomarkers,Total bilirubin ≤1.5 x institutional upper lim...,ULN,closest,inclusion,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,C92172,biomarkers,EGFR Activating Mutation,exclusion,NaN,C3910,biomarkers,Molecular Abnormality,True,NaN
9,9,NCT02991651,Inclusion Criteria:\n* Patients must have hist...,C180730,prior_therapy,EGFR tyrosine kinase inhibitor,Epidermal Growth Factor Receptor Inhibitor-Ind...,closest,exclusion,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
from functools import partial, wraps

import psycopg2 as pg

con = pg.connect(dbname="sec", user="secapp", host="r_363-postgres-1", port=5432)

In [3]:
def with_cursor(func=None, *, con=None):
    if func is None:
        return partial(with_cursor, con=con)

    @wraps(func)
    def wrapper(*args, **kwargs):
        cur = con.cursor()
        res = None
        try:
            res = func(*args, **kwargs, cur=cur)
        except Exception as e:
            con.rollback()
            # print(e)
            # print("closing gracefully")
            cur.close()
        else:
            # print("closing")
            cur.close()
        return res

    return wrapper


In [4]:
@with_cursor(con=con)
def get_shared_parent(concept1, concept2, cur=None):
    if pd.isna([concept1, concept2]).any():
        return []
    cur.execute(
        """select *
    from
    (
        with p1 as (
            select
                distinct parent, level
            from
                ncit_tc_with_path
            where
                descendant = %s
        ),
        p2 as ( 
            select
                distinct parent, level
            from
                ncit_tc_with_path
            where
                descendant = %s
        )
        select
                p1.parent,
                p1.level + p2.level as num_hops,
                dense_rank() over(order by p1.level + p2.level) as rank
        from p1
        join p2 on p2.parent = p1.parent
        order by num_hops
    ) t
    where rank <= 3""",
        vars=(concept1, concept2),
    )
    rows = cur.fetchall()
    return rows

In [5]:
get_shared_parent("C136482", "C4012")

[('C4878', 5, 1),
 ('C2916', 7, 2),
 ('C7377', 7, 2),
 ('C4571', 9, 3),
 ('C9305', 9, 3),
 ('C3200', 9, 3),
 ('C3576', 9, 3),
 ('C3709', 9, 3)]

In [6]:
import math

# Separate dataframes by llm vs cts api
df_llm = df.loc[df["llm_nct_id"].notna(), ["llm_ncit_code", "llm_type"]]
df_cts = df.loc[df["ctsapi_nct_id"].notna(), ["ctsapi_ncit_code", "ctsapi_type"]]

# Separate dataframes by LLM biomarkers, diseases, and prior therapies
llm_markers = df_llm[df_llm["llm_type"] == "biomarkers"]
llm_dise = df_llm[df_llm["llm_type"] == "diseases"]
llm_pt = df_llm[df_llm["llm_type"] == "prior_therapy"]

# Separate dataframes by  CTS biomarkers, diseases, and prior therapies
cts_markers = df_cts[df_cts["ctsapi_type"] == "biomarkers"]
cts_dise = df_cts[df_cts["ctsapi_type"] == "diseases"]
cts_pt = df_cts[df_cts["ctsapi_type"] == "prior_therapy"]

# Inner (cross) Join LLM and CTS scientific concepts
xjoin_markers = pd.merge(
    llm_markers,
    cts_markers,
    how="inner",
    left_on="llm_type",
    right_on="ctsapi_type",
)[["llm_ncit_code", "ctsapi_ncit_code"]]
xjoin_dise = pd.merge(
    llm_dise,
    cts_dise,
    how="inner",
    left_on="llm_type",
    right_on="ctsapi_type",
)[["llm_ncit_code", "ctsapi_ncit_code"]]
xjoin_pt = pd.merge(
    llm_pt,
    cts_pt,
    how="inner",
    left_on="llm_type",
    right_on="ctsapi_type",
)[["llm_ncit_code", "ctsapi_ncit_code"]]


def process_similar_concepts(row: pd.Series):
    if row.llm_ncit_code == row.ctsapi_ncit_code:
        return 0
    shared_parents = get_shared_parent(row.llm_ncit_code, row.ctsapi_ncit_code)
    if shared_parents:
        # print(shared_parents[0])
        return shared_parents[0][1]
    else:
        return math.inf


# Save the distances from each biomaker in LLM to each biomarker in CTS
xjoin_markers["dist_away"] = xjoin_markers.apply(process_similar_concepts, axis=1)
xjoin_pt["dist_away"] = xjoin_pt.apply(process_similar_concepts, axis=1)
xjoin_dise["dist_away"] = xjoin_dise.apply(process_similar_concepts, axis=1)

In [7]:
# See what the dataframe looks like before being cross joined
print(
    df[(df["llm_type"] == "diseases") | (df["ctsapi_type"] == "diseases")][
        ["llm_ncit_code", "ctsapi_ncit_code"]
    ].sort_values(["llm_ncit_code", "ctsapi_ncit_code"], ascending=False)
)


   llm_ncit_code ctsapi_ncit_code
18         C7783            C7783
19         C4012              NaN
17         C3813              NaN
20           NaN          C136482


In [8]:
# Group by LLM NCIt code for further transformation on dist_away column
xjoin_dise_dist = xjoin_dise.groupby("llm_ncit_code")["dist_away"]
# Save the minimum dist for the NCIt code group
xjoin_dise = xjoin_dise.assign(min_dist=xjoin_dise_dist.transform(min))

# Flatten the LLM NCIt code group into a single code, min dist pair
llm_mins = xjoin_dise.groupby("llm_ncit_code")["min_dist"].min()
# Flatten the CTS NCIt code group into a single code, min dist pair
cts_mins = xjoin_dise.groupby("ctsapi_ncit_code")["dist_away"].min()

# Start preparing new rows for the dataframe using the LLM NCIt code, min dist into
# these leftrows will be joined to the left section (LLM) of the original df
leftrows = []
for key, val in llm_mins.items():
    for idx, row in xjoin_dise[
        (xjoin_dise["llm_ncit_code"] == key) & (xjoin_dise["dist_away"] == val)
    ][["llm_ncit_code", "ctsapi_ncit_code", "min_dist"]].iterrows():
        d = {
            "left": row["llm_ncit_code"],
            "right": row["ctsapi_ncit_code"],
            "dist": row["min_dist"],
        }
        leftrows.append(d)
# Start preparing new rows for the dataframe using the CTS NCIt code, min dist into
# these rightrows will be joined to the right section (CTS) of the original df
rightrows = []
for key, val in cts_mins.items():
    for idx, row in xjoin_dise[
        (xjoin_dise["ctsapi_ncit_code"] == key) & (xjoin_dise["dist_away"] == val)
    ][["llm_ncit_code", "ctsapi_ncit_code", "dist_away"]].iterrows():
        d = {
            "right": row["ctsapi_ncit_code"],
            "left": row["llm_ncit_code"],
            "dist": row["dist_away"],
        }
        rightrows.append(d)
new_df_left = pd.DataFrame(leftrows).drop_duplicates()
new_df_right = pd.DataFrame(rightrows).drop_duplicates()
# If the same code has equal distances to other codes in the comparison set,
# join them using a comma
left = new_df_left.groupby(["left", "dist"])["right"].aggregate(lambda x: ",".join(x))
left = left.to_frame().reset_index()
right = new_df_right.groupby(["right", "dist"])["left"].aggregate(lambda x: ",".join(x))
right = right.to_frame().reset_index()

In [9]:
left

,left,dist,right
0,C3813,8,"C7783,C136482"
1,C4012,3,C7783
2,C7783,0,C7783


In [10]:
right

,right,dist,left
0,C136482,4,C7783
1,C7783,0,C7783


In [11]:
# Rejoin the distance dataframes to the original
final_df = (
    pd.merge(df, left, how="left", left_on="llm_ncit_code", right_on="left")
    .drop(["left"], axis=1)
    .rename(columns={"right": "ctsapi_closest", "dist": "dist_to_cts"})
)
final_df = (
    pd.merge(final_df, right, how="left", left_on="ctsapi_ncit_code", right_on="right")
    .drop(["right"], axis=1)
    .rename(columns={"left": "llm_closest", "dist": "dist_to_llm"})
)

In [12]:
final_df = final_df[
    [
        "Column1",
        "llm_nct_id",
        "llm_eligibility_criteria",
        "llm_ncit_code",
        "llm_type",
        "ctsapi_closest",
        "dist_to_cts",
        "llm_extracted_term",
        "llm_selected_term",
        "llm_selection_proximity",
        "llm_eligibility_criterion",
        "llm_invalid_ncit_code",
        "llm_expected_branch_code",
        "llm_expected_branch_type",
        "llm_expected_branch_name",
        "llm_type_matches_branch_type",
        "ctsapi_nct_id",
        "ctsapi_eligibility_criteria",
        "ctsapi_ncit_code",
        "llm_closest",
        "dist_to_llm",
        "ctsapi_type",
        "ctsapi_term",
        "ctsapi_eligibility_criterion",
        "ctsapi_branch",
        "ctsapi_expected_branch_code",
        "ctsapi_expected_branch_type",
        "ctsapi_expected_branch_name",
        "ctsapi_type_matches_branch_type",
        "ctsapi_branch_matches_branch_name",
    ]
]
final_df.to_csv('tmp.csv')